[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_garch_electricity/EMQA_garch_electricity.ipynb)

# EMQA_garch_electricity
GARCH(1,1) on electricity returns -- higher and spikier volatility than oil.
**Output:** `garch_electricity.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
url = 'https://raw.githubusercontent.com/QuantLet/EMQA/main/EMQA_garch_electricity/electricity_cache.csv'
elec = pd.read_csv(url, parse_dates=[0], index_col=0)
if 'price' in elec.columns:
    price_col = 'price'
elif 'Price' in elec.columns:
    price_col = 'Price'
else:
    price_col = elec.columns[0]
elec_price = elec[price_col].dropna()
print(f'Loaded {len(elec_price)} electricity price observations')


In [ ]:
# Compute daily returns from electricity prices
elec_ret = elec_price.pct_change().dropna() * 100
# Winsorize extreme outliers for numerical stability
clip_val = elec_ret.abs().quantile(0.995)
elec_ret = elec_ret.clip(-clip_val, clip_val)
print(f"Electricity returns: {len(elec_ret)} obs, std={elec_ret.std():.2f}%")

In [ ]:
# Fit GARCH(1,1)
try:
    from arch import arch_model
    am = arch_model(elec_ret, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off')
    print(res.summary())
    cond_vol = res.conditional_volatility
except ImportError:
    print("arch not available -- simulating GARCH")
    omega, alpha, beta_ = 0.5, 0.15, 0.80
    n = len(elec_ret)
    cond_var = np.zeros(n)
    cond_var[0] = elec_ret.var()
    ret_vals = elec_ret.values
    for t in range(1, n):
        cond_var[t] = omega + alpha * ret_vals[t-1]**2 + beta_ * cond_var[t-1]
    cond_vol = pd.Series(np.sqrt(np.maximum(cond_var, 0)), index=elec_ret.index)

In [ ]:
# 2-panel plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1 - Returns with bands
axes[0].plot(elec_ret.index, elec_ret.values, color=COLORS['gray'], linewidth=0.4, alpha=0.7, label='Returns')
axes[0].plot(cond_vol.index, 2 * cond_vol.values, color=COLORS['red'], linewidth=1, label='+2$\\sigma$')
axes[0].plot(cond_vol.index, -2 * cond_vol.values, color=COLORS['red'], linewidth=1, label='-2$\\sigma$')
axes[0].set_ylabel('Returns (%)')
axes[0].set_title('Electricity Returns - GARCH(1,1) Conditional Bands')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=3)

# Panel 2 - Conditional volatility
axes[1].plot(cond_vol.index, cond_vol.values, color=COLORS['orange'], linewidth=1)
axes[1].set_ylabel('Conditional Volatility (%)')
axes[1].set_xlabel('Date')
axes[1].set_title('GARCH(1,1) Conditional Volatility - Electricity')

plt.tight_layout()
save_fig(fig, 'garch_electricity.pdf')
plt.show()